<h1><b>Capstone Project - The Battle of the Neighborhoods (Week2)</b></h1>

<a>1.Introduction: Business Problem</a><br>
<a>2.Datasets</a><br>
<a>3.Methodology</a><br>
<a>4.Analysis</a><br>
<a>5.Results and Discussion</a><br>
<a>6.Conclusion</a><br>

<h3>1.Introduction: Business Problem</h3>

In [6]:
!conda install -c conda-forge geopy 
!conda install -c conda-forge folium=0.5.0 

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.3              |   py37h5e8e339_2         

In [ ]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium


Load data about postal codes

In [12]:
#read the data from the json file ("laposte_hexasmal")
datap = pd.read_json('https://datanova.laposte.fr/explore/dataset/laposte_hexasmal/download/?format=json')
datap

AttributeError: 'NoneType' object has no attribute 'items'

              datasetid                                  recordid  \
0      laposte_hexasmal  3c535b8fd874a8801f1f1441e49020b1bb19c3ab   
1      laposte_hexasmal  60ec8e153fd1dae8db09308c95dcef7e7a29836e   
2      laposte_hexasmal  a499405b7273bf33265de31f3e8c8f770857b737   
3      laposte_hexasmal  d7af047b53af8f8e8c8a8518bb14a88aa7882b5b   
4      laposte_hexasmal  75b4293f4938479498a2bc8ffbd1a84668481d67   
...                 ...                                       ...   
39187  laposte_hexasmal  db10289bd074cc9cf3317b96e4ae20f4001d94b4   
39188  laposte_hexasmal  97719f28a9dc42fd448347b2fb6dc01ad4003e16   
39189  laposte_hexasmal  a3007754b816e668bb432530ce63aa0e6bf363b1   
39190  laposte_hexasmal  bf472305e66faf4949e9808bcb7e32f9c2e0590b   
39191  laposte_hexasmal  85b8b659fd308cc5f33097dd8546edea530a75cd   

                                                  fields  \
0      {'code_commune_insee': '01004', 'nom_de_la_com...   
1      {'code_commune_insee': '01008', 'nom_de_la_c

In [14]:
data_paris=datap['fields']
dataframe = json_normalize(data_paris)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


AttributeError: 'NoneType' object has no attribute 'items'

  code_commune_insee  nom_de_la_commune code_postal  \
0              01004  AMBERIEU EN BUGEY       01500   
1              01008           AMBUTRIX       01500   
2              01015    ARBOYS EN BUGEY       01300   
3              01017              ARGIS       01230   
4              01019              ARMIX       01510   

                  coordonnees_gps libelle_d_acheminement  ligne_5  
0   [45.9608475114, 5.3729257777]      AMBERIEU EN BUGEY      NaN  
1   [45.9367134524, 5.3328092349]               AMBUTRIX      NaN  
2  [45.7237621545, 5.65263086429]        ARBOYS EN BUGEY  ST BOIS  
3  [45.9337182132, 5.48251100314]                  ARGIS      NaN  
4  [45.8541838459, 5.58357838695]                  ARMIX      NaN  

Select just the features revelant for this project 'code_postal' and 'coordonnees_gps' and clean data, wrangling data

In [15]:
# print Size of dataframe
filtered_columns = ['code_postal'] + ['coordonnees_gps']
dataframe_filtered = dataframe.loc[:, filtered_columns]
print('Size of dataframe is:',dataframe_filtered.shape)
dataframe_filtered

Size of dataframe is: (39192, 2)


AttributeError: 'NoneType' object has no attribute 'items'

      code_postal                 coordonnees_gps
0           01500   [45.9608475114, 5.3729257777]
1           01500   [45.9367134524, 5.3328092349]
2           01300  [45.7237621545, 5.65263086429]
3           01230  [45.9337182132, 5.48251100314]
4           01510  [45.8541838459, 5.58357838695]
...           ...                             ...
39187       98824                             NaN
39188       98825                             NaN
39189       98882                             NaN
39190       98831                             NaN
39191       98799                             NaN

[39192 rows x 2 columns]

In [16]:
dataframe_clean=dataframe_filtered.dropna()
dataframe_clean.reset_index(drop=True)
print('Size of dataframe is:',dataframe_clean.shape)
dataframe_clean

Size of dataframe is: (38922, 2)


AttributeError: 'NoneType' object has no attribute 'items'

      code_postal                  coordonnees_gps
0           01500    [45.9608475114, 5.3729257777]
1           01500    [45.9367134524, 5.3328092349]
2           01300   [45.7237621545, 5.65263086429]
3           01230   [45.9337182132, 5.48251100314]
4           01510   [45.8541838459, 5.58357838695]
...           ...                              ...
39130       97426  [-21.1104753596, 55.3297522865]
39131       97620  [-12.9244813606, 45.0984953615]
39132       97660  [-12.8534223602, 45.1755961092]
39133       97615  [-12.7791666355, 45.2822648358]
39134       97615  [-12.7961353309, 45.2842063102]

[38922 rows x 2 columns]

In [17]:
dataframe_clean['lat'],dataframe_clean['lng']=zip(*dataframe_clean.pop('coordonnees_gps'))
dataframe_clean

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


AttributeError: 'NoneType' object has no attribute 'items'

      code_postal        lat        lng
0           01500  45.960848   5.372926
1           01500  45.936713   5.332809
2           01300  45.723762   5.652631
3           01230  45.933718   5.482511
4           01510  45.854184   5.583578
...           ...        ...        ...
39130       97426 -21.110475  55.329752
39131       97620 -12.924481  45.098495
39132       97660 -12.853422  45.175596
39133       97615 -12.779167  45.282265
39134       97615 -12.796135  45.284206

[38922 rows x 3 columns]

In [18]:
df_Paris=dataframe_clean[dataframe_clean['code_postal'].str.startswith('750')].reset_index(drop=True)
print('Number of district in Paris :', df_Paris.count())
df_Paris

Number of district in Paris : code_postal    20
lat            20
lng            20
dtype: int64


AttributeError: 'NoneType' object has no attribute 'items'

   code_postal        lat       lng
0        75004  48.854228  2.357362
1        75007  48.856083  2.312439
2        75011  48.859415  2.378741
3        75015  48.840155  2.293559
4        75020  48.863187  2.400820
5        75006  48.848968  2.332671
6        75012  48.835156  2.419807
7        75013  48.828718  2.362468
8        75018  48.892735  2.348712
9        75002  48.867903  2.344107
10       75003  48.863054  2.359361
11       75008  48.872527  2.312583
12       75014  48.828993  2.327101
13       75016  48.860399  2.262100
14       75017  48.887337  2.307486
15       75019  48.886869  2.384694
16       75001  48.862630  2.336293
17       75005  48.844509  2.349859
18       75009  48.876896  2.337460
19       75010  48.876029  2.361113

<h3>Foursquare</h3>
Define Foursquare Credentials and Version in hidden cell

In [27]:
CLIENT_ID = '2TPWJYHGDGU0F3O0UZIZOMD0QJR2ZBLPDMJJYEQ1I2UARBIJ' 
CLIENT_SECRET = 'FT1MTLKTJKN1OVXJLHKIYZBPDJIXZ4EOSYNHHSKNVXRJAPPV' 

VERSION = '20180604' 
LIMIT = 100 
RADIUS=500

<h3>Explore Neighborhoods in Paris</h3>

In [28]:
# Category IDs corresponding to Hotel category were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
hotel_category = '4bf58dd8d48988d1fa931735' # 'Root' category for all hotels-related venues
metro_category = '4bf58dd8d48988d1fd931735' # 'Root' category for metro station
bus_category = '52f2ab2ebcbc57f1066b8b4f' # 'Root' category for bus stop
museum_category='4bf58dd8d48988d181941735'
restaurant_category='4d4b7105d754a06374d81259'
bar_category='4d4b7105d754a06376d81259'
parking_category='4c38df4de52ce0d596b336e1'

def get_hotels_count_near_location(latitude, longitude, category):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&oauth_token={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET, VERSION,ACCESS_TOKEN, latitude, longitude, category, RADIUS,LIMIT)
    jSon=requests.get(url).json()
    return (jSon['response']['totalResults'])

<h3>Add a new columns :</h3

In [29]:
for index in df_Paris.index:
    print(df_Paris.loc[index]['code_postal'])
    df_Paris.loc[index,'Number_Hotels']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],hotel_category)
    df_Paris.loc[index,'Numbers_Metro']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],metro_category)
    df_Paris.loc[index,'Number_Bus_Stop']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],bus_category)
    df_Paris.loc[index,'Number_Museum']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],museum_category)
    df_Paris.loc[index,'Number_Restaurant']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],restaurant_category)
    df_Paris.loc[index,'Number_Bar']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],bar_category)
    df_Paris.loc[index,'Number_Parking']=get_hotels_count_near_location(df_Paris.loc[index]['lat'],df_Paris.loc[index]['lng'],parking_category)

75004


KeyError: 'totalResults'

In [30]:
print('List column name of dataframe :', df_Paris.columns.values)
df_Paris.describe()

List column name of dataframe : ['code_postal' 'lat' 'lng']


AttributeError: 'NoneType' object has no attribute 'items'

             lat        lng
count  20.000000  20.000000
mean   48.860290   2.344437
std     0.018763   0.037009
min    48.828718   2.262100
25%    48.847853   2.323471
50%    48.861515   2.346410
75%    48.873403   2.361452
max    48.892735   2.419807

In [31]:
#We creat a dataframe with top 10 attractions in Paris
top= {
    'Name_atraction' : ['Musee_Orsay', 'Sainte_Chapelle', 'Palais_Garnier', 'Notre_Dame', 'Musee_Orangerie', 'Luxembourg_Gardens', 'Louvre', 'Eiffel', 'Pont_Alexandre', 'Marais'],
    'Latitude' : [48.859961,48.855375,48.871970,48.852968, 48.863788,48.846222,48.860611,48.858370,48.863900,48.861233],
    'Longitude' : [2.326561,2.344961,2.331601,2.349902,2.322672,2.337160,2.337644,2.294481,2.313559,2.358189],
}    
df_top=pd.DataFrame(top)
print('Data dimension of the resulting dataframe:', df_top.shape)

Data dimension of the resulting dataframe: (10, 3)


In [32]:
# we will define function get_distance that calculate the distance between two coordonates
from math import sin, cos, sqrt, atan2, radians
def get_distance(lat,lng,Latitude,Longitude,inKm=True,precision=2):
    # approximate radius of earth in km
    R = 6373.0
    if inKm==False: #we want in meters so adjust the radius to meters
        R=R*1000
    
    #convert degrees to radians -> Deg × π/180
    lat1 = radians(lat)
    lon1 = radians(lng)
    lat2 = radians(Latitude)
    lon2 = radians(Longitude)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return round(R * c, precision)

calculate the distance from our location candidate and the top 10 attractions

In [33]:
for index in df_Paris.index:
    #print(df_Paris.loc[index]['code_postal'])
    for index2 in df_top.index:
        #print("--",df_top.loc[index2]['Name_atraction'])
        df_Paris.loc[index, "DistanceTo."+df_top.loc[index2]['Name_atraction']   ]=get_distance(df_Paris.loc[index]['lat'],
                                                    df_Paris.loc[index]['lng'],
                                                    df_top.loc[index2]['Latitude'],
                                                    df_top.loc[index2]['Longitude'])
df_Paris

AttributeError: 'NoneType' object has no attribute 'items'

   code_postal        lat       lng  DistanceTo.Musee_Orsay  \
0        75004  48.854228  2.357362                    2.34   
1        75007  48.856083  2.312439                    1.12   
2        75011  48.859415  2.378741                    3.82   
3        75015  48.840155  2.293559                    3.27   
4        75020  48.863187  2.400820                    5.45   
5        75006  48.848968  2.332671                    1.30   
6        75012  48.835156  2.419807                    7.36   
7        75013  48.828718  2.362468                    4.36   
8        75018  48.892735  2.348712                    3.99   
9        75002  48.867903  2.344107                    1.56   
10       75003  48.863054  2.359361                    2.42   
11       75008  48.872527  2.312583                    1.73   
12       75014  48.828993  2.327101                    3.44   
13       75016  48.860399  2.262100                    4.72   
14       75017  48.887337  2.307486                    

In [34]:
df_Paris.insert(10,"Total_distance",df_Paris.apply(lambda row: row[11 : ].sum(),axis=1))
df_Paris.columns.values

AttributeError: 'Series' object has no attribute '_mgr'

<h5>Let's create a map centered around Paris and play with the zoom level to see how it affects the top location map</h5>

In [37]:
import folium 
paris_map = folium.Map(location=[48.859489, 2.320582], zoom_start=12)


# instantiate a feature group for the incidents in the dataframe
Paris_group = folium.map.FeatureGroup()
# loop through the district and add each to the incidents feature group
#set the color of the location either yeallow (below average number of hotels or red for above average)
for lat, lng, label,hotel in zip(df_Paris.lat, df_Paris.lng, df_Paris.code_postal, df_Paris.Number_Hotels):
    outercolor='blue'
    if hotel > 27:  #27 being the mean number of hotels
        outercolor='red'
    Paris_group.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=outercolor,
            popup=("{}: number hotel {}").format(label,int(hotel)),
            fill=True,
            fill_color='yellow',
            fill_opacity=0.6
        )
    )

# add incidents to map
paris_map.add_child(Paris_group)

# instantiate a feature group for the top 10 attraction in Paris
Paris_top10 = folium.map.FeatureGroup()
# loop through the district and add each to the incidents feature group
for lat, lng, label in zip(df_top.Latitude, df_top.Longitude, df_top.Name_atraction):
    Paris_top10.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='green',
            popup=label,
            fill=True,
            #fill_color='red',
            fill_opacity=1
        )
    )

# add incidents to map
paris_map.add_child(Paris_top10)

# add incidents to map
paris_map.add_child(Paris_group)
   
paris_map

AttributeError: 'DataFrame' object has no attribute 'Number_Hotels'

<h3>3.Methodology</h3>


In this project we will focus to make an optimal recommendation for the site selection of a hotel.<br>
My master data will have the main components: list of districts with chosen features and location of attractions.<br>
In our analysis will do the calculation and exploration of venues across each districts of Paris.<br>
    We will used python folium library to visualize geographic details of Paris.<br>
    We will utilize the Foursquare API to explore the districts and segment them.<br>
    We will analyze the link between the number of hotels and the distance to attractions (heatmaps, correlation,...) to help focus our attention on ideal areas<br>
    We will used unsupervised learning K-means algorithm to cluster the districts.<br>
    We will present map all potential locations but also create clusters (using k-means clustering) to identify the optimal recommendation for the site.<br>
    We will used also, GitHub as our repository.</br>
In the first step in our analysis we have collected the distance from our potential locations to top attractions in Paris, the availability of existing hotels, restaurants, bars and accessibility to public transport and parking. We also cleaned, selected and mapped the features we are interested in. The second step in our analysis will be exploring our datasets, we will identify the relationships between our features. In the third step we will focus on creating clusters (using k-means clustering) of those locations and their features to identify the optimal districts for our potential location. Finally, we will identify and optimal propose location to stakeholders

<h3>4.Analysis</h3>

Now let's calculate two most important things for each location candidate:<br>

the relationship between number of hotels in vicinity and the distance to the top attractions<br>
the relationship between the potential location, total distance, numbers restaurants and numbers of bars<br>

In [38]:
from numpy import cov
cov_hot_dist = cov(df_Paris['Number_Hotels'], df_Paris['Total_distance'])
print('Covariance between Number_Hotels and Total_distance : '), cov(df_Paris['Number_Hotels'], df_Paris['Total_distance'])

KeyError: 'Number_Hotels'

In [39]:
import seaborn as sns 
import matplotlib
import matplotlib.pyplot as plt
sns.regplot(x='Number_Hotels', y='Total_distance', data=df_Paris)
plt.ylim(0,)

ValueError: variadic keyword parameters cannot have default values

In [ ]:
from scipy.stats import pearsonr
from scipy import stats
pearson_coef, p_value  = stats.pearsonr(df_Paris['Number_Hotels'], df_Paris['Total_distance'])
print('The Pearson Correlation is', pearson_coef, 'with a P_value of P =', p_value)

Conclusion:Since the p-value is <0.001, the correlation between (Number_Hotels and Total_distance) is statistically significant, and the negative linear relationship is quite strong (~-0.717).
<br>Plotting relationship between the total distance, numbers restaurants and numbers of bars and potential location

In [ ]:
#we plot the data to see the relationship between the features:
df_plot=df_Paris[['code_postal','Number_Hotels','Total_distance','Number_Restaurant','Number_Bar']].sort_values(['code_postal'], ascending=True, axis=0)
df_plot.set_index('code_postal', inplace=True)
ax=df_plot.plot(kind='bar', 
                  figsize=(20, 8), 
                  width=0.8, 
                  color=('#98534f','#5cb85c', '#5bc0de', '#d9534f'), 
                  fontsize=14)
ax.set_title("Features determinate our potential location", 
             fontsize = 16)
ax.legend(fontsize = 14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)  
#for p in ax.patches:
#    ax.annotate("{:.2%}".format(p.get_height()),
#                xy=(p.get_x()+0.02, p.get_height()+0.01))

In [ ]:
df_Paris[['Number_Hotels', 'Numbers_Metro', 'Numbers_Metro',
       'Number_Bus_Stop', 'Number_Museum', 'Number_Restaurant',
       'Number_Bar', 'Number_Parking', 'Total_distance']].describe().round(decimals=3)

In [ ]:
print('Average distance to closest Hotel from each top:', df_Paris['Total_distance'].mean())

In [ ]:
#order our dataframe by Total_distance
df_Paris.sort_values(['Total_distance'], ascending=True, axis=0, inplace=True)
df_distance=df_Paris[['code_postal', 'Total_distance']]
df_distance

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# sphinx_gallery_thumbnail_number = 2


tran = ['Numbers_Metro','Number_Bus_Stop','Number_Parking']
far = df_Paris['code_postal'].to_list()

harvest = df_Paris[['Numbers_Metro','Number_Bus_Stop','Number_Parking']].transpose().values


fig, ax = plt.subplots(figsize=(12, 8))

#fig.tight_layout()
#plt.show()

im = ax.imshow(harvest)

# We want to show all ...
ax.set_xticks(np.arange(len(far)))
ax.set_yticks(np.arange(len(tran)))
# ... and label them with the respective list entries
ax.set_xticklabels(far)
ax.set_yticklabels(tran)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(tran)):
    for j in range(len(far)):
        text = ax.text(j, i, harvest[i, j],
                       ha="center", va="center", color="w")

ax.set_title("The transport accesibility from potential location")
#fig.tight_layout()
plt.show()

Relationship between multiple variables at the same time

In [ ]:
#Correlation matrix to get a table containing the correlation coefficients between each variable and the others.
df_small=df_Paris[['Number_Hotels',
       'Numbers_Metro', 'Number_Bus_Stop', 'Number_Museum',
       'Number_Restaurant', 'Number_Bar', 'Number_Parking',
       'Total_distance']].corr()
df_small
sns.heatmap(df_small, annot=True)
plt.show()

<h3>Modeling</h3>

In [ ]:
Optimising K - Elbow Method

In [ ]:
from sklearn import metrics
from scipy.spatial.distance import cdist

df_Paris_k = df_Paris[['Number_Hotels', 'Numbers_Metro',
       'Number_Bus_Stop', 'Number_Museum', 'Number_Restaurant',
       'Number_Bar', 'Number_Parking', 'Total_distance',
       'DistanceTo.Musee_Orsay', 'DistanceTo.Sainte_Chapelle',
       'DistanceTo.Palais_Garnier', 'DistanceTo.Notre_Dame',
       'DistanceTo.Musee_Orangerie', 'DistanceTo.Luxembourg_Gardens',
       'DistanceTo.Louvre', 'DistanceTo.Eiffel',
       'DistanceTo.Pont_Alexandre', 'DistanceTo.Marais']].values

In [ ]:
from scipy.spatial.distance import cdist
distortions = []
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_Paris_k)
    distortions.append(sum(np.min(cdist(df_Paris_k, km.cluster_centers_, 
                      'euclidean'),axis=1)) / df_Paris_k.shape[0]) 
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(K, distortions, 'bx-')
ax2.plot(K, Sum_of_squared_distances, 'bx-')
ax1.set_title('The Elbow Method using Distortion', fontsize = 15)
ax2.set_title('The Elbow Method using Inertia', fontsize = 15)
ax1.set_ylabel('Distortion', fontsize = 12)
ax2.set_ylabel('Sum_of_squared_distances', fontsize = 12)
ax1.set_xlabel('k', fontsize = 12)
ax2.set_xlabel('k', fontsize = 12)

We deduce that k=3 is the optimal K

<h5>Clustering using KMeans<h5>

In [ ]:
from sklearn.cluster import KMeans

number_of_clusters = 3

kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(df_Paris_k)

kmeans.labels_
#grab the labels for each point in the model using Kmean

In [8]:
#Insert column Cluster Labels in the first position of dataframe
df_Paris.insert(0, 'Cluster Labels', kmeans.labels_)

NameError: name 'df_Paris' is not defined

In [ ]:
df_Paris

In [ ]:
df_Paris2=df_Paris[['Cluster Labels','code_postal','Number_Hotels','Total_distance']].sort_values(['Cluster Labels'], ascending=True, axis=0).reset_index(drop=True)
df_Paris2

Creating a Choropleth map

In [ ]:
df_choropleth=df_Paris[['code_postal','Cluster Labels']].sort_values('Cluster Labels').reset_index(drop=True) 
df_choropleth=df_choropleth.astype(int)
df_choropleth['code_postal']=df_choropleth['code_postal']+100
df_choropleth

In [ ]:

#to load the geoJson file for Paris arrondissment
paris_geo = r'C:/Users/ruxan/Desktop/GABI/CERTIFICATION/IBM-DATASCIENT/IBM/Capstone-9/project-week4-5/arrondissements.geojson' # geojson file

visualise the clusters using an interactive Plotly map

In [ ]:
# we create the map of Paris and color each district by label
paris_map = folium.Map(location=[48.859489, 2.320582], zoom_start=12)
paris_map.choropleth(
    geo_data=paris_geo,
    data=df_choropleth,
    columns=['code_postal','Cluster Labels', 'Number_Hotels'],
    key_on='feature.properties.c_arinsee',
    threshold_scale=range(number_of_clusters+1),
    fill_color='PuOr', 
    fill_opacity=0.8, 
    line_opacity=0.2,
    legend_name='Cluster Labels from locations in Paris',
    reset=True
)

paris_map.add_child(Paris_group)
paris_map.add_child(Paris_top10)
paris_map

<h3>Examine Clusters</h3>

Cluster_0

In [ ]:
df_Paris_1=df_Paris.loc[df_Paris['Cluster Labels'] == 0, df_Paris.columns[list(range(4, 12))]]
df_Paris_1.loc['mean']=df_Paris_1.mean()
for col in df_Paris_1.columns:
   df_Paris_1[col]= df_Paris_1[col].map(lambda x: '%.2f' % x)

df_Paris_1

Cluster_1

In [ ]:
df_Paris_2=df_Paris.loc[df_Paris['Cluster Labels'] == 1, df_Paris.columns[list(range(4, 12))]]
df_Paris_2.loc['mean']=df_Paris_2.mean()
#to show the dataframe with decimal we use a loop
for col in df_Paris_2.columns:
   df_Paris_2[col]= df_Paris_2[col].map(lambda x: '%.2f' % x)


df_Paris_2

Cluster_2

In [ ]:
df_Paris_3=df_Paris.loc[df_Paris['Cluster Labels'] == 2, df_Paris.columns[list(range(4, 12))]]
df_Paris_3.loc['mean']=df_Paris_3.mean()
for col in df_Paris_3.columns:
   df_Paris_3[col]= df_Paris_3[col].map(lambda x: '%.2f' % x)

df_Paris_3

In [ ]:
The principal's features that will influence our selection of Cluster are as the total distance to the top of attractions and Number Hotels around our potential location to be minimum

In [ ]:

#Creat a new dataframe of Means of each cluster
df_means=pd.DataFrame()
for i in range(number_of_clusters):
    df_means = df_means.append(df_Paris.loc[df_Paris['Cluster Labels'] == i,['Cluster Labels','Number_Hotels','Total_distance']].mean(), ignore_index=True)
df_means.set_index('Cluster Labels', inplace=True)
df_means.round(decimals=3)

In [ ]:
df_means[df_means['Total_distance']==df_means['Total_distance'].min()]

In [ ]:
df_means[df_means['Number_Hotels']==df_means['Number_Hotels'].min()]

In [ ]:
loc_paris=df_Paris.loc[df_Paris['Cluster Labels'] != 0, df_Paris.columns[[1]+list(range(4, 12))]]
loc_paris

In [ ]:
#Create dataframe of transport related feature in the selected group of potential location
loc_paris_1_2_tr=loc_paris[['code_postal','Numbers_Metro', 'Number_Bus_Stop', 'Number_Parking']].set_index('code_postal')
loc_paris_1_2_tr['Numbers_Metro_over_mean'] = loc_paris_1_2_tr.apply(lambda x: x['Numbers_Metro'] >= int(loc_paris_1_2_tr['Numbers_Metro'].mean()), axis=1)
loc_paris_1_2_tr['Number_Bus_Stop_over_mean'] = loc_paris_1_2_tr.apply(lambda x: x['Number_Bus_Stop'] >= int(loc_paris_1_2_tr['Number_Bus_Stop'].mean()), axis=1)
loc_paris_1_2_tr['Number_Parking_over_mean'] = loc_paris_1_2_tr.apply(lambda x: x['Number_Parking'] >= int(loc_paris_1_2_tr['Number_Parking'].mean()), axis=1)


loc_paris_1_2_tr

In [ ]:

print('Mean of Numbers_Metro is :', loc_paris_1_2_tr['Numbers_Metro'].mean())
print('Mean of Number_Bus_Stop is:',loc_paris_1_2_tr['Number_Bus_Stop'].mean())
print('Mean of Number_Parking is:', loc_paris_1_2_tr['Number_Parking'].mean())

In [ ]:

#3.Histogram of the transport features that determinate our potential location
ax=loc_paris_1_2_tr.plot(kind='bar', 
                  figsize=(20, 8), 
                  width=0.8, 
                  color=("#542788","#8073ac","#e08214" ), 
                  fontsize=14)
ax.set_title("Transport features determinate our potential location", 
             fontsize = 16)
ax.legend(fontsize = 14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

In [ ]:
#2.Create dataframe of Foods and Drinks features (Restaurant/bars) in the selected group of potential location
loc_paris_1_2_bar=loc_paris[['code_postal','Number_Restaurant','Number_Bar']].set_index('code_postal')
loc_paris_1_2_bar['number_Restaurant_over_mean'] = loc_paris_1_2_bar.apply(lambda x: x['Number_Restaurant'] >= int(loc_paris_1_2_bar['Number_Restaurant'].mean()), axis=1)
loc_paris_1_2_bar['Number_Bar_over_mean'] = loc_paris_1_2_bar.apply(lambda x: x['Number_Bar'] >= int(loc_paris_1_2_bar['Number_Bar'].mean()), axis=1)
loc_paris_1_2_bar

In [ ]:
#4.Histogram of the Foods and Drinks features (Restaurant/bars) that determinate our potential location
ax=loc_paris_1_2_bar.plot(kind='bar', 
                  figsize=(20, 8), 
                  width=0.8, 
                  color=("#d8daeb","#998ec3","#f7f7f7","#d8daeb","#b2abd2","#8073ac" ), 
                  fontsize=14)
ax.set_title("Features Foods and drinks determinate our potential location", 
             fontsize = 16)
ax.legend(fontsize = 14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

In [ ]:
print('Mean of Number_Restaurant is :', loc_paris_1_2_bar['Number_Restaurant'].mean())
print('Mean of Number_Bar is:',loc_paris_1_2_bar['Number_Bar'].mean())

<h3>5.Results and Discussion</h3>

Our analysis shows that beside the fact there is a great number of hotels in Paris, there are places for hotels close to city center.<br>
We targeted the tourists that want to explore the attractions and enjoy French cuisine and night life. The highest concentration of hotels was detected in the first 10 district (at the center of Paris), but also the total distance to the principal top attractions in these districts is lower the mean of 30 km, also in plus is a low of density of restaurants, bars and very good connexions in Paris.
<br>After analysing the relations between the features, we found that our potential districts (postal code) have a positive correlations between the transport connections, number hotels and negative correlations between total distance directing our attention to this more narrow area of interest, taking as primarily feature total distance, so close to attraction and second number of hotels.
<br>Thescation candidates were then clustered to create zones of interest which contain fewer number of locations. I used unsupervised learning K-means algorithm to cluster the districts. K-Means algorithm is one of the most common clustering methods of unsupervised learning.
<br>First, I will run K-Means to cluster the districts into 3 clusters because analyzing the K-Means with Elbow method had ensured us the 3 degree is the optimum k of the K-Means.
<br>We visualise the clusters using an interactive Plotly map that doesn’t indicate the possible location, but it indicates that there is a higher density of top attractions in cluster 1, also they have more the average of number of hotel (bullet red outer color). Moreover, Cluster 2 is furthest from the attractions, but they also have the least number of hotels.
<br>The principal features that will influence our selection of Cluster are the total distance to the top attractions and the Number of Hotels around our potential location to be minimum. Based on this we will now focus our analysis on areas of Cluster 1 (close to attractions) and Cluster 2 (least number of hotels).
<br>Examining our clusters also, we found the cluster 1 and 2 are the best location To select our possible location, we will analyse the others features. We split those features in two group (Transport and Foods & Drinks). After plot the histograms we found that 75001 is the better optional for transport and second for restaurants.
<br>Purpose of this analysis was to make a recommendation where will be good for opening an hotel having as target the tourists.  

<h3>6.Conclusion</h3>

Purpose of this project was to identify Paris areas close to top attractions with a low number of hotels which would be an optimal location for a new hotel.
<br>Using Foursquare data, we visualise the clusters using an interactive Plotly map and we identify general districts that justify further analysis of cluster 1.
<br>I use machine learning and use Kmean to help us to choose the better location. We selected cluster 1 and 2 as the best locations because of the negative correlation between number of hotels and distance to attractions we keep the minimum of these two features.
<br>We explore cluster data frame 1 and 2 in order to create a major zone of interest and visually inspect the selected Clusters, plotting data.
<br>Final decision on optimal restaurant location will be made by investors based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to attraction), proximity to metro, parking, bars, restaurants